In [1]:
from nltk.stem import LancasterStemmer

def stemmer_func(word_to_stem):

    lancaster = LancasterStemmer()

    return lancaster.stem(word_to_stem)

In [13]:
from nltk.corpus import wordnet

def find_syn(word_seeking):
    synonyms = []

    for syn in wordnet.synsets(word_seeking):
        for each in syn.lemmas():
            synonyms.append(each.name())
    return synonyms

print(set(find_syn("account")))

{'write_up', 'score', 'business_relationship', 'bill', 'history', 'account_statement', 'accounting', 'chronicle', 'calculate', 'invoice', 'describe', 'story', 'explanation', 'report', 'news_report', 'answer_for', 'account'}


In [35]:
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet

lemmatizer = WordNetLemmatizer()

print(lemmatizer.lemmatize("Policemen"))


import spacy


nlp = spacy.load('en_core_web_sm')

doc = nlp("I am a Policemen.")

for token in doc:
    print(token.dep_, token.text)

Policemen
nsubj I
ROOT am
det a
attr Policemen
punct .


In [3]:
import pandas as pd

def load_csv(csv_file):
    return pd.read_csv(csv_file)

job_data = load_csv("seek_australia.csv")

job_data.head()

,category,city,company_name,geo,job_board,job_description,job_title,job_type,post_date,salary_offered,state,url
0,Retail & Consumer Products,Sydney,Frontline Executive Retail Sydney,AU,seek,Have you had 10 years experience in fresh pro...,Store Manager - Fresh Produce,Full Time,2018-04-15T23:13:45Z,$100k Base + Super + Benefits,North Shore & Northern Beaches,https://www.seek.com.au/job/35989382
1,Government & Defence,Brisbane,Powerlink,AU,seek,The Opportunity: The Client Solution Analyst ...,Client Solution Analyst,Full Time,2018-04-15T23:04:40Z,Excellent remuneration packages,Northern Suburbs,https://www.seek.com.au/job/35989272
2,Trades & Services,Sydney,Richard Jay Laundry,AU,seek,An innovative business development role for a...,Service Technician / Installer - NSW,Full Time,2018-04-15T23:04:31Z,NaN,Parramatta & Western Suburbs,https://www.seek.com.au/job/35989270
3,Trades & Services,Melbourne,Adaptalift Hyster,AU,seek,About the role: We are seeking an Automotive W...,Workshop Technician I Material Handling Equipment,Full Time,2018-04-16T03:15:17Z,NaN,Bayside & South Eastern Suburbs,https://www.seek.com.au/job/35993203
4,Trades & Services,Adelaide,Bakers Delight G&M,AU,seek,Â Early starts and weekend shifts. No experie...,APPRENTICESHIP JUNIOR BAKER,Full Time,2018-04-16T01:26:50Z,NaN,NaN,https://www.seek.com.au/job/35991578


In [4]:
def get_search_list(token_text):
    search_list = []
    search_list.append(token_text)
    search_list.append(stemmer_func(token_text))
    search_list.extend(find_syn(token_text))
    search_list.extend(find_syn(stemmer_func(token_text)))
                       
    return set(search_list)
    
# print(get_search_list("Accountant"))

In [22]:
import spacy
from spacy.matcher import Matcher
from spacy import displacy
from nltk.corpus import wordnet

nlp = spacy.load('en_core_web_sm')

def get_result(doc, csv_file):
    

    # doc = nlp("Show me a job for Accountant in Sydney.")

    nlp = spacy.load(u'en_core_web_sm')

    #job_data = []
    
    city_column_num = 0
    url_column_num = 0
    job_data = load_csv(csv_file)
    csv_headers = list(job_data.columns)
    # print(csv_headers)
    for each in range(len(csv_headers)):
        if csv_headers[each].lower() == 'city':
            city_column_num = each
        #print(each_job[each].lower())
        if csv_headers[each].lower() == 'url':
            url_column_num = each   
            
    #print(city_column_num, url_column_num)

    '''
    with open("seek_australia.csv", 'r', encoding='utf-8') as f:
        flag = 0
        for each in f:
            
            each_job = each.replace('\n','').split(',')
            #print(each_job)
            if flag == 0:
                flag = 1
                for each in range(len(each_job)):
                    if each_job[each].lower() == 'city':
                        city_column_num = each
                    #print(each_job[each].lower())
                    if each_job[each].lower() == 'url':
                        url_column_num = each     
                    
            else:
                job_data.append(each_job)
    '''
    matching_list = []
    for token in doc:
        #print('text: {}, \t ent_type_: {}, \t head: {}, \t dep_: {}'.format(token.text, token.ent_type_, token.head, token.dep_))

        if token.head.dep_ == 'prep':
            #print('I am here, I am {}.'.format(token))
            #print(stemmer_func(token.text))
            matching_list.append(token)

    result_list = []
    #print(url_column_num)
    #print(type(matching_list[0]))

    search_list = get_search_list(matching_list[0].text)

    #print(search_list)
    row_num = 0
    for row in job_data.itertuples(index = True):
        break_flag = 0
        for each in search_list:
            for every in csv_headers:
                # print(each)
                if each in str(getattr(row, every)) and str(getattr(row, csv_headers[city_column_num])) == matching_list[1].text:
                    result_list.append([row_num, each, every])
                    break_flag = 1
                    break
                    
            if break_flag:
                break

        row_num += 1
        

    '''
    for each in range(len(job_data)):
        flag_break = 0
        for every in range(len(job_data[each])):
            for per in search_list:
                if per in job_data[each][every] and job_data[each][city_column_num] == matching_list[1].text:
                    # print(job_data[each])
                    #print(job_data[each][city_column_num])
                    #result_list.append([each, every, per]) # uncommand this to find the matching cell
                    #result_list.append(each)
                    #print(job_data[each][url_column_num])
                    for _ in job_data[each]:
                        print(_, end="\n\n\n\n\n")
                    result_list.append(job_data[each][url_column_num])
                    flag_break = 1
                    break
            if flag_break:
                break
    '''
    return result_list

print(get_result(nlp("Show me a job for Accountant in Sydney."), "seek_australia.csv"))

3786


In [ ]:
import spacy
from telegram.ext import Updater, MessageHandler, Filters
#the callback function that uses spaCy
def utterance(update, context):
    msg = update.message.text
    nlp = spacy.load('en_core_web_sm')
    doc = nlp(msg)
    '''
    for token in doc:
        if token.dep_ == 'dobj':
            update.message.reply_text('We are processing your request...')
            return
    '''
    #print(return_destination(doc), return_destination(doc).ent_type_)
        
    if matching_it(doc) == True:
        update.message.reply_text('I am fine thank you and you?')

    elif return_destination(doc)[0].ent_type_ == 'GPE':
        # print(1)
        update.message.reply_text('It seems you want a ticket from {} to {}. Alright, below is the ticket of the destination place you want to go'.format(return_destination(doc)[0].text, return_destination(doc)[1].text))
    else:
        update.message.reply_text('Please rephrase your request. Be as specific as possible!')

#the code responsible for interactions with Telegram
updater = Updater(TOKEN, use_context=True)
updater.dispatcher.add_handler(MessageHandler(Filters.text, utterance))
updater.start_polling()
updater.idle()

In [43]:
spacy.explain("prep")

'prepositional modifier'

In [108]:
from nltk.stem import LancasterStemmer

lancaster = LancasterStemmer()

print(lancaster.stem("Accountant"))

account


In [103]:
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize

ps = PorterStemmer()

print(ps.stem("fireman"))

fireman


In [107]:
from nltk.stem.snowball import SnowballStemmer

stemmer = SnowballStemmer("english")

print(stemmer.stem("accountant"))

account


In [10]:
## 尝试复杂的句子，entity1

import spacy
from spacy.matcher import Matcher
from spacy import displacy
from nltk.corpus import wordnet

nlp = spacy.load('en_core_web_sm')

doc = nlp("I work in taxation for ten years, show me a job in Sydney.")

matching_list = []
for token in doc:
    print('text: {},  ent_type_: {}, child: {}, head: {},  dep_: {},  pos_: {} '.format(token.text, token.ent_type_, list(token.children), token.head, token.dep_, token.pos_))

    if token.head.dep_ == 'prep':

        matching_list.append(token)
        
print(matching_list)

text: I,  ent_type_: , child: [], head: work,  dep_: nsubj,  pos_: PRON 
text: work,  ent_type_: , child: [I, in, for, ,, show, .], head: work,  dep_: ROOT,  pos_: VERB 
text: in,  ent_type_: , child: [taxation], head: work,  dep_: prep,  pos_: ADP 
text: taxation,  ent_type_: , child: [], head: in,  dep_: pobj,  pos_: NOUN 
text: for,  ent_type_: , child: [years], head: work,  dep_: prep,  pos_: ADP 
text: ten,  ent_type_: DATE, child: [], head: years,  dep_: nummod,  pos_: NUM 
text: years,  ent_type_: DATE, child: [ten], head: for,  dep_: pobj,  pos_: NOUN 
text: ,,  ent_type_: , child: [], head: work,  dep_: punct,  pos_: PUNCT 
text: show,  ent_type_: , child: [me, job], head: work,  dep_: conj,  pos_: VERB 
text: me,  ent_type_: , child: [], head: show,  dep_: dative,  pos_: PRON 
text: a,  ent_type_: , child: [], head: job,  dep_: det,  pos_: DET 
text: job,  ent_type_: , child: [a, in], head: show,  dep_: dobj,  pos_: NOUN 
text: in,  ent_type_: , child: [Sydney], head: job,  d

In [9]:
spacy.explain('PROPN')

'proper noun'

In [36]:
document = nlp("2+ years experience working in a social media,")

for token in document:
    print(token.text, token.pos_, token.dep_)

2 NUM nummod
+ NUM amod
years NOUN compound
experience NOUN ROOT
working VERB acl
in ADP prep
a DET det
social ADJ amod
media NOUN pobj
, PUNCT punct


In [27]:
spacy.explain('PROPN') 

'proper noun'

In [11]:
import spacy
from collections import Counter

nlp = spacy.load("en_core_web_sm")

def word_freq(text):
    doc = nlp(text)
    words = []
    
    for each in doc:
        if not each.is_stop and not each.is_punct:
            words.append(each.text)

    freq = Counter(words)
    common = freq.most_common()
    return common

word_freq("I have a pen. I have an apple. Emmm, apple pen.")

[('pen', 2), ('apple', 2), ('Emmm', 1)]

In [2]:
import pandas as pd

new_df = pd.read_csv('finished1.txt', delimiter='|')

In [9]:
new_df.head()

,Unnamed: 0,category,city,company_name,geo,job_board,job_description,job_title,job_type,post_date,salary_offered,state,url,tokenised,lemmatize,stemmed,retokened,cleaned,tfidf
0,0,Retail & Consumer Products,Sydney,Frontline Executive Retail Sydney,AU,seek,Have you had years experience in fresh produc...,Store Manager - Fresh Produce,Full Time,2018-04-15T23:13:45Z,$100k Base + Super + Benefits,North Shore & Northern Beaches,https://www.seek.com.au/job/35989382,"['Have', 'you', 'had', 'years', 'experience', ...","['have', 'you', 'had', 'years', 'experience', ...","{'own', 'button', 'and', 'retail', 'use', 'k',...","['service brand', 'shopping otherwise manager ...","['service brand', 'shopping otherwise manager ...","(array(['amazing', 'ambitious', 'attitude', 'a..."
1,1,Government & Defence,Brisbane,Powerlink,AU,seek,The Opportunity: The Client Solution Analyst p...,Client Solution Analyst,Full Time,2018-04-15T23:04:40Z,Excellent remuneration packages,Northern Suburbs,https://www.seek.com.au/job/35989272,"['The', 'Opportunity', 'The', 'Client', 'Solut...","['the', 'opportunity', 'the', 'client', 'solut...","{'button', 'and', 'ani', 'function', 'technic'...",['resume standards information working powerli...,['resume standards information working powerli...,"(array(['ability', 'account', 'accounts', 'acc..."
2,2,Trades & Services,Sydney,Richard Jay Laundry,AU,seek,An innovative business development role for an...,Service Technician / Installer - NSW,Full Time,2018-04-15T23:04:31Z,NaN,Parramatta & Western Suburbs,https://www.seek.com.au/job/35989270,"['An', 'innovative', 'business', 'development'...","['an', 'innovative', 'business', 'development'...","{'and', 'global', 'suit', 'save', 'the', 'give...","['equipment laundries', 'working warehouses', ...","['equipment laundries', 'working warehouses', ...","(array(['adelaide', 'agencies', 'an', 'applica..."
3,3,Trades & Services,Melbourne,Adaptalift Hyster,AU,seek,About the role: We are seeking an Automotive W...,Workshop Technician I Material Handling Equipment,Full Time,2018-04-16T03:15:17Z,NaN,Bayside & South Eastern Suburbs,https://www.seek.com.au/job/35993203,"['About', 'the', 'role', 'We', 'are', 'seeking...","['about', 'the', 'role', 'we', 'are', 'seeking...","{'own', 'and', 'sent', 'ani', 'learn', 'techni...","['bcarey@aalgroup.com.au', 'meet', 'belinda em...","['bcarey@aalgroup.com.au', 'meet', 'belinda em...","(array(['aalgroup', 'aalhysterforklifts', 'abo..."
4,4,Trades & Services,Adelaide,Bakers Delight G&M,AU,seek,Early starts and weekend shifts. No experience...,APPRENTICESHIP JUNIOR BAKER,Full Time,2018-04-16T01:26:50Z,NaN,NaN,https://www.seek.com.au/job/35991578,"['Early', 'starts', 'and', 'weekend', 'shifts'...","['early', 'starts', 'and', 'weekend', 'shifts'...","{'own', 'potenti', 'and', 'store', 'shift', 'n...","['apprenticeship', 'long you', 'store experien...","['apprenticeship', 'long you', 'store experien...","(array(['apprenticeship', 'attitude', 'experie..."


In [16]:
def retokeniser(text):
    doc = nlp(text)
    with doc.retokenize() as retokenizer:
        for each in doc.noun_chunks:
            retokenizer.merge(each)
            doc.retokenize().merge(each)
            
    return doc

In [26]:
import pandas as pd

def load_csv(csv_file):
    return pd.read_csv(csv_file)

job_data = load_csv("seek_australia.csv")

In [27]:
# try to remove the broken characters

for each in ['š', 'ª', 'º', 'Ÿ', 'µ', 'ˆ', 'œ', 'é', 'è', 'ï', 'Š', 'â', 'ç', 'æ', 'å', 'ä', 'ð', 'Ã', 'Â', 'Å', 'Ä', '®', '€', '™', '\xa0', '\d+']:
    try:
        job_data['job_description_clean'] = job_data['job_description_clean'].replace({each:''}, regex=True)
    except:
        job_data['job_description_clean'] = job_data['job_description'].replace({each:''}, regex=True)

In [36]:
import nltk

vocab_list = []

for each in job_data['job_description_clean']:
    try:
        doc = retokeniser(each)

        list_1 = [tok.text for idx,tok in enumerate(doc)]
        
        vocab_list.extend(list_1)
    except:
        pass

np_fd = nltk.FreqDist(vocab_list)

print(len(vocab_list))

print(len(np_fd))




6969677
531418


In [39]:
count = 0

for key, value in zip(np_fd.keys(), np_fd.values()):
    if value > 30:
        count += 1
        print(key)
        
        
count

 
Have
you
had
 years experience
in
that
wants
to
manage
for
is
passionate
about
food
.
We
are
looking
:
Must
have
+ years
and
the passion
the role
progress
with
training
into
Be
hands
on
a
can
do
attitude
,
ambitious
competitive
people
who
Are
able
maximise
their market
ensuring
it
meets
sales
margin
wages
budgets
customer service skills
helping
us
create
our customers
a wealth
of
knowledge
a willingness
share
this knowledge
Can
lead
motivate
be
work
weekend
use
which
all
What
s
+
super
benefits
opportunities
Strong
expanding
business
If
this
apply
now
interviewing
To
online
please
click
the apply button
Alternatively
a confidential discussion
contact
quoting
Ref No
or
otherwise
check
out
our website
other available positions
www.frontlineretail.com.au
The Opportunity
provides
technical support
through
receiving
responding
queries
relating
technical expertise
issues
This role
ensures
the delivery
customers
Key Accountabilities
Manager
user
accounts
including
the maintenance
Manage
inc

10240

In [40]:
nltk.download('stopwords')

stop_words = set(nltk.corpus.stopwords.words("english"))

corpus_list = []



[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/zhanghanlin/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
